# Prédiction des tags

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", None)

datas = pd.read_csv("../P5_06_transf_datas.csv")

datas

Je sépare le jeu de données en un jeu d'entraînement et un de test.

In [ ]:
X_train = datas["imp_words"]
Y_train = datas["Tags"]

In [ ]:
# samples = np.random.choice(len(X_train), 5000, replace=False)
# X_train = X_train[samples]
# Y_train = Y_train[samples]

Je  transforme X et y.

In [ ]:
# from sklearn.base import TransformerMixin
# from sklearn.feature_extraction.text import CountVectorizer

# class TfidfVectorsToArray(TransformerMixin):
#     def fit(self, X, y=None, **fit_params):
#         return self

#     def transform(self, X, y=None, **fit_params):
#         return X.toarray()

# class YToBagOfWords(CountVectorizer):
#     def fit(self, X= None, y= y_train, **fit_params):
#         return super().fit(y)

#     def transform(self, X= None, y= y_train, **fit_params):
#         return super().transform(y)

#     def fit_transform(self, X= None, y= y_train, **fit_params):
#         return super().fit_transform(y)

# class YToArray(TransformerMixin):
#     def fit(self, X= None, y= y_train, **fit_params):
#         return self

#     def transform(self, X= None, y= y_train, **fit_params):
#         return y.toarray()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from P5_custom_module import TfidfVectorsToArray

pipeline = Pipeline(steps=[
    # ("count_vect", CountVectorizer(token_pattern= r'[\S]+')),
    ("tfidf", TfidfVectorizer(token_pattern= r'[\S]+')),
    ("toarray", TfidfVectorsToArray()),
    ("PCA", PCA(n_components= 1500)),
    ("clf", OneVsRestClassifier(
        SGDClassifier(loss= "modified_huber", class_weight= None, 
        penalty= "l1", alpha= 0.0001, l1_ratio= 0.15, early_stopping= True, 
        random_state= 42)))
    ])

# count_vect = CountVectorizer(token_pattern= r'[\S]+').fit_transform(X_train)
# x_train = TfidfTransformer().fit_transform(count_vect)

# tfidf_vect = TfidfVectorizer(token_pattern= r'[\S]+')

# x_train = tfidf_vect.fit_transform(X_train).toarray()


In [ ]:
# component = 1500

# pca = PCA(n_components= component)
# pca.fit(x_train)

# pca.explained_variance_ratio_.cumsum()[-1]

In [ ]:
# x_train_pca = pca.transform(x_train)


In [ ]:
count_vect = CountVectorizer(token_pattern= r'[\S]+', min_df= 10)
y_train = count_vect.fit_transform(Y_train).toarray()


In [ ]:
with open("tags.txt", "w") as file:
    file.write(" ".join(count_vect.get_feature_names_out()))

In [ ]:
# y_train.shape

'score': 0.9919314591700134,
 'loss': 'modified_huber',
 'class_weight': None,
 'penalty': 'l1',
 'alpha': 0.0001,
 'l1_ratio': 0.15

In [ ]:
# from sklearn.multiclass import OneVsRestClassifier
# from sklearn.linear_model import SGDClassifier

# clf = OneVsRestClassifier(SGDClassifier(loss= "modified_huber", class_weight= None, 
#     penalty= "l1", alpha= 0.0001, l1_ratio= 0.15, early_stopping= True, 
#     random_state= 42))

pipeline.fit(X_train, y_train)

In [ ]:
pred = pipeline.predict_proba(["c# don't open mysql database with windows access file "])

In [ ]:
# pred.shape

In [ ]:
results_preds = np.zeros(pred.shape)

pred_sorted = np.sort(pred, axis= 1)
diff = pred_sorted[:, -1] - np.log10(pred_sorted[:, -1] +1)

results_preds = np.where(pred > diff.reshape(-1, 1), 1, 0)

In [ ]:
# results_preds.shape

In [ ]:
with open("tags.txt", "r") as file:
    cols = file.read().split()


In [ ]:
# cols = count_vect.get_feature_names_out()

results_df = pd.DataFrame(results_preds, columns= cols).T.reset_index()
" ".join(results_df.loc[results_df[0] > 0, "index"].values)

Sauvegarde du modèle

In [ ]:
import joblib

# joblib.dump(pipeline, "P5_tags_prediction.sav")